## POS 데이터 제작

In [356]:
from konlpy.tag import Mecab
import numpy as np
import pandas as pd
from tqdm import tqdm
mecab = Mecab()

In [357]:
# score t가 있는 파일
scoret = pd.read_csv('./df_origin_jae.csv')

In [358]:
paragraph_df = pd.read_csv('./paragraph.csv')

In [359]:
paragraph_df

,paragraph_txt
0,나는 다수결을 반대한대 #@문장구분# 왜냐하면 이거를 싫어하는사람도분명히 있을텐데 ...
1,교육을 받기를 원하는 사람이 있으면 딱히 받고 싶지 않은 사람이 있을 지도 모른다....
2,남한과 북한의 통일은 너무너무 복잡한 일이라고 생각합니다. #@문장구분# 우리는 북...
3,요즘 환경문제에 대한 뉴스를 쉽게 접할 수 있습니다. #@문장구분# 최근에는 미국 ...
4,나는남북 통일에찬성 합니다. #@문장구분# 왜냐하면 전쟁을하다가 죽은사람이 얼마나 ...
...,...
5995,내가 최근에 봤던 영화는 블랙 위도우라는 마블 영화이다. #@문장구분# 나는 원래 ...
5996,제가 다녔던 유치원은 초등학교 부설 유치원이어서 초등학교에 입학하지 않았어도 방과후...
5997,나는 옛날부터 it관련된 직업에 관심이 많이 있었었다. #@문장구분# 그래서 it관...
5998,저의 꿈은 초등학교 선생님입니다. #@문장구분# 스승이라는 단어를 생각하면 대다수가...


In [360]:
# 열 추가
paragraph_df['sentence_list'] = [None]*6000
paragraph_df['sentence_count'] = [None]*6000
paragraph_df['pos_tagging'] = [None]*6000
paragraph_df['pos_list'] = [None]*6000
paragraph_df['punc_sentence_list'] = [None]*6000
paragraph_df['punc_pos_tagging'] = [None]*6000
paragraph_df['punc_pos_list'] = [None]*6000
paragraph_df['punctuation_marks'] = [None]*6000
paragraph_df['ending_of_a_word'] = [None]*6000
paragraph_df['word_order'] = [None]*6000

In [362]:
for i in tqdm(range(len(paragraph_df))):
    paragraph_df['sentence_list'][i]=paragraph_df['paragraph_txt'][i].split('#@문장구분#')
    paragraph_df['sentence_count'][i] = len(paragraph_df['sentence_list'][i])
    punc_spc = {'."':'',  '?"':'', '!"':'', ".'":'', "?'":'', "!'":'',  "!,":'', ".,":'', "?,":''}
    original = paragraph_df['paragraph_txt'][i]
    
    for p in punc_spc:
        original = original.replace(p, punc_spc[p])
        
    paragraph_df['punc_sentence_list'][i] = original.split('#@문장구분#')

paragraph_df.head(3)

100%|██████████| 6000/6000 [00:02<00:00, 2409.87it/s]


,paragraph_txt,sentence_list,sentence_count,pos_tagging,pos_list,punc_sentence_list,punc_pos_tagging,punc_pos_list,punctuation_marks,ending_of_a_word,word_order
0,나는 다수결을 반대한대 #@문장구분# 왜냐하면 이거를 싫어하는사람도분명히 있을텐데 ...,"[나는 다수결을 반대한대 , 왜냐하면 이거를 싫어하는사람도분명히 있을텐데 다 수결...",7,None,None,"[나는 다수결을 반대한대 , 왜냐하면 이거를 싫어하는사람도분명히 있을텐데 다 수결...",None,None,None,None,None
1,교육을 받기를 원하는 사람이 있으면 딱히 받고 싶지 않은 사람이 있을 지도 모른다....,[교육을 받기를 원하는 사람이 있으면 딱히 받고 싶지 않은 사람이 있을 지도 모른다...,11,None,None,[교육을 받기를 원하는 사람이 있으면 딱히 받고 싶지 않은 사람이 있을 지도 모른다...,None,None,None,None,None
2,남한과 북한의 통일은 너무너무 복잡한 일이라고 생각합니다. #@문장구분# 우리는 북...,"[남한과 북한의 통일은 너무너무 복잡한 일이라고 생각합니다. , 우리는 북한과 오...",9,None,None,"[남한과 북한의 통일은 너무너무 복잡한 일이라고 생각합니다. , 우리는 북한과 오...",None,None,None,None,None


In [363]:
# POS Tagging
for i in tqdm(range(len(paragraph_df))):
    sentence_pos_list = [] # pos된 것 list  [[('나', 'NP'), ('는', 'JX'), ('다수결', 'NNG'),
    pos_list = [] # tag만 담긴거 list [[NP, JX, NNG ... 
    
    for sentence in paragraph_df['sentence_list'][i]:
        poss = []
        tagging = mecab.pos(sentence)
        
        for tags in tagging:
            poss.append(tags[1])
            
        sentence_pos_list.append(tagging)
        pos_list.append(poss)
        
    paragraph_df['pos_tagging'][i] = sentence_pos_list
    paragraph_df['pos_list'][i] = pos_list
    
    # punc
    punc_sentence_pos_list = [] # pos된 것 list  [[('나', 'NP'), ('는', 'JX'), ('다수결', 'NNG'),
    punc_pos_list = [] # tag만 담긴거 list [[NP, JX, NNG ... 
    for sentence in paragraph_df['punc_sentence_list'][i]:
        poss = []
        tagging = mecab.pos(sentence)
        
        for tags in tagging:
            poss.append(tags[1])
            
        punc_sentence_pos_list.append(tagging)
        punc_pos_list.append(poss)
        
    paragraph_df['punc_pos_tagging'][i] = punc_sentence_pos_list
    paragraph_df['punc_pos_list'][i] = punc_pos_list

100%|██████████| 6000/6000 [00:19<00:00, 303.03it/s]


In [365]:
paragraph_df.head(3)

,paragraph_txt,sentence_list,sentence_count,pos_tagging,pos_list,punc_sentence_list,punc_pos_tagging,punc_pos_list,punctuation_marks,ending_of_a_word,word_order
0,나는 다수결을 반대한대 #@문장구분# 왜냐하면 이거를 싫어하는사람도분명히 있을텐데 ...,"[나는 다수결을 반대한대 , 왜냐하면 이거를 싫어하는사람도분명히 있을텐데 다 수결...",7,"[[(나, NP), (는, JX), (다수결, NNG), (을, JKO), (반대,...","[[NP, JX, NNG, JKO, NNG, XSA+ETM, NNG], [MAJ, ...","[나는 다수결을 반대한대 , 왜냐하면 이거를 싫어하는사람도분명히 있을텐데 다 수결...","[[(나, NP), (는, JX), (다수결, NNG), (을, JKO), (반대,...","[[NP, JX, NNG, JKO, NNG, XSA+ETM, NNG], [MAJ, ...",None,None,None
1,교육을 받기를 원하는 사람이 있으면 딱히 받고 싶지 않은 사람이 있을 지도 모른다....,[교육을 받기를 원하는 사람이 있으면 딱히 받고 싶지 않은 사람이 있을 지도 모른다...,11,"[[(교육, NNG), (을, JKO), (받, VV), (기, ETN), (를, ...","[[NNG, JKO, VV, ETN, JKO, VV, ETM, NNG, JKS, V...",[교육을 받기를 원하는 사람이 있으면 딱히 받고 싶지 않은 사람이 있을 지도 모른다...,"[[(교육, NNG), (을, JKO), (받, VV), (기, ETN), (를, ...","[[NNG, JKO, VV, ETN, JKO, VV, ETM, NNG, JKS, V...",None,None,None
2,남한과 북한의 통일은 너무너무 복잡한 일이라고 생각합니다. #@문장구분# 우리는 북...,"[남한과 북한의 통일은 너무너무 복잡한 일이라고 생각합니다. , 우리는 북한과 오...",9,"[[(남한, NNG), (과, JC), (북한, NNP), (의, JKG), (통일...","[[NNG, JC, NNP, JKG, NNG, JX, MAG, XR, XSA+ETM...","[남한과 북한의 통일은 너무너무 복잡한 일이라고 생각합니다. , 우리는 북한과 오...","[[(남한, NNG), (과, JC), (북한, NNP), (의, JKG), (통일...","[[NNG, JC, NNP, JKG, NNG, JX, MAG, XR, XSA+ETM...",None,None,None


## 문장 부호

In [366]:
# 틀린 것을 저장할 배열
check_punc = [0]*len(paragraph_df)
for i in tqdm(range(len(paragraph_df))): # 6000행 검사
    for j,pos_list in enumerate(paragraph_df['punc_pos_list'][i]): # 문장마다 검사
        # 맨 마지막이 문장부호로 끝나는지 검사한다.
        if pos_list == []:
            continue
        if pos_list[-1] != 'SF' and pos_list[-1]!=[]:
            check_punc[i]+=1
        
        # 중복된 문장기호가 있는지 검사한다.
        for k in range(len(pos_list)): # 문장에 있는 태그마다 검사            
            if pos_list[k]=='SF': # 마침표 물음표 느낌표
                if len(pos_list)-1!=k: # 맨마지막이 아니라면

                    if pos_list[k+1]=='SY':
                        check_punc[i]+=1
                        
    paragraph_df['punctuation_marks'][i] = check_punc[i]
    
        # 확인용
#         if check_punc[i]>=7:
#             print(paragraph_df['punc_pos_tagging'][i])
#             print()

100%|██████████| 6000/6000 [00:01<00:00, 5821.67it/s]


In [367]:
print(check_punc[500:800])

[0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 3, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 2, 3, 0, 0, 0, 1, 0, 0, 5, 0, 0, 0, 6, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 10, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]


In [370]:
paragraph_df.tail(2)

,paragraph_txt,sentence_list,sentence_count,pos_tagging,pos_list,punc_sentence_list,punc_pos_tagging,punc_pos_list,punctuation_marks,ending_of_a_word,word_order
5998,저의 꿈은 초등학교 선생님입니다. #@문장구분# 스승이라는 단어를 생각하면 대다수가...,"[저의 꿈은 초등학교 선생님입니다. , 스승이라는 단어를 생각하면 대다수가 제일 ...",12,"[[(저, NP), (의, JKG), (꿈, NNG), (은, JX), (초등, N...","[[NP, JKG, NNG, JX, NNG, NNG, NNG, XSN, VCP+EF...","[저의 꿈은 초등학교 선생님입니다. , 스승이라는 단어를 생각하면 대다수가 제일 ...","[[(저, NP), (의, JKG), (꿈, NNG), (은, JX), (초등, N...","[[NP, JKG, NNG, JX, NNG, NNG, NNG, XSN, VCP+EF...",0,None,None
5999,몇 시간만에 이 소설을 딱 읽고 나서는 참 비참하다고 생각하였다. #@문장구분# 얼...,"[몇 시간만에 이 소설을 딱 읽고 나서는 참 비참하다고 생각하였다. , 얼마나 비...",24,"[[(몇, MM), (시간, NNG), (만, NNB), (에, JKB), (이, ...","[[MM, NNG, NNB, JKB, MM, NNG, JKO, MAG, VV, EC...","[몇 시간만에 이 소설을 딱 읽고 나서는 참 비참하다고 생각하였다. , 얼마나 비...","[[(몇, MM), (시간, NNG), (만, NNB), (에, JKB), (이, ...","[[MM, NNG, NNB, JKB, MM, NNG, JKO, MAG, VV, EC...",1,None,None


## 어미

In [371]:
# 틀린 것을 저장할 배열
check_end = [0]*len(paragraph_df)
for i in tqdm(range(len(paragraph_df))): # 6000행 검사
    find_last = -1
    S_flag = 0 # 종결 부호 없이 종료된 경우 EF가 아니라 EC (연결 어미)라고 뜬다.

    for j,pos_list in enumerate(paragraph_df['punc_pos_list'][i]): # 문장마다 검사
        if pos_list == []:
            continue

        #print(paragraph_df['punc_pos_list'][i])

        S_flag = 0 # 종결 부호 없이 종료된 경우 EF가 아니라 EC (연결 어미)라고 뜬다.
        try:
            # 종결 부호라면..
            if (pos_list[find_last] == 'SF' or pos_list[find_last]=='SY') and pos_list[find_last]!=[]:
                find_last -= 1
                # 종결 부호 없이 종료된 경우
                if pos_list[find_last]=='SF':
                    S_flag=1
            
            # 종결어미 or 다른것+종결어미 형태가 아니라면, 종결부호 없이 종료된 경우 연결어미인 인 경우가 아니라면
            elif pos_list[find_last][-2:] != 'EF' or(S_flag ==1 and pos_list[find_last][-2:] != 'EC'): 
                check_end[i]+=1
                
        except IndexError:
            pass
                        
    paragraph_df['ending_of_a_word'][i] = check_end[i]

    
#     #확인용
#     if check_end[i]>=12:
#         print(paragraph_df['punc_pos_tagging'][i])
#         print(i)
#         print()

100%|██████████| 6000/6000 [00:00<00:00, 8085.63it/s]


In [373]:
print(check_end[500:800])

[0, 0, 0, 0, 0, 10, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 2, 0, 7, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 1, 0, 0, 0, 0, 6, 15, 0, 0, 0, 2, 1, 1, 0, 2, 0, 1, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 5, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 3, 2, 3, 0, 0, 0, 1, 0, 0, 6, 0, 0, 0, 6, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 3, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 10, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0]


## 어순
- 주격조사 앞에는 체언이 있어야한다 JKS, N으로시작
- 보격조사 앞에는 체언이 있어야한다.JKC, N으로 시작
- 목적격조사 + 명사인 경우는 안된다.JKO + N으로 시작

In [374]:
# 틀린 것을 저장할 배열

check_order = [0]*len(paragraph_df)
for i in tqdm(range(len(paragraph_df))): # 6000행 검사
    for j,pos_list in enumerate(paragraph_df['punc_pos_list'][i]): # 문장마다 검사
        
        if pos_list == []:
            continue
        
        # 목적격조사 뒤에 **?(일반명사를 제외한) 명사가 있는가 검사 # mecab: ('를', 'JKO'), ('말', 'NNG'), ('한다면', 'XSV+EC')...
        for k in range(len(pos_list)): # 문장에 있는 태그마다 검사            
            if pos_list[k]=='JKO': # 목적격조사JKO
                if len(pos_list)-1!=k: # 맨마지막이 아니라면
                    if pos_list[k+1][0]=='N':# and pos_list[k+1]!='NNG':
                        check_order[i]+=1
        
        # 주격조사 앞에는 체언이 있어야한다.
        for k in range(len(pos_list)): # 문장에 있는 태그마다 검사            
            if pos_list[k]=='JKS': # 주격조사 JKS
                if len(pos_list)-1!=0: # 맨 앞이 아니라면
                    if pos_list[k-1][0]!='N':# 체언이 아니라면
                        check_order[i]+=1
            
         # 보격조사 앞에는 체언이 있어야한다.
        for k in range(len(pos_list)): # 문장에 있는 태그마다 검사            
            if pos_list[k]=='JKC': # 보격조사 JJKC
                if len(pos_list)-1!=0: # 맨 앞이 아니라면
                    if pos_list[k-1][0]!='N':# 체언이 아니라면
                        check_order[i]+=1                       
                        
    paragraph_df['word_order'][i] = check_order[i]
    
    
    # 확인용
#     if check_order[i]>=7:
#         print(paragraph_df['punc_pos_tagging'][i])
#         print()

100%|██████████| 6000/6000 [00:01<00:00, 4377.46it/s]


In [376]:
paragraph_df.tail(1)

,paragraph_txt,sentence_list,sentence_count,pos_tagging,pos_list,punc_sentence_list,punc_pos_tagging,punc_pos_list,punctuation_marks,ending_of_a_word,word_order
5999,몇 시간만에 이 소설을 딱 읽고 나서는 참 비참하다고 생각하였다. #@문장구분# 얼...,"[몇 시간만에 이 소설을 딱 읽고 나서는 참 비참하다고 생각하였다. , 얼마나 비...",24,"[[(몇, MM), (시간, NNG), (만, NNB), (에, JKB), (이, ...","[[MM, NNG, NNB, JKB, MM, NNG, JKO, MAG, VV, EC...","[몇 시간만에 이 소설을 딱 읽고 나서는 참 비참하다고 생각하였다. , 얼마나 비...","[[(몇, MM), (시간, NNG), (만, NNB), (에, JKB), (이, ...","[[MM, NNG, NNB, JKB, MM, NNG, JKO, MAG, VV, EC...",1,3,4


In [377]:
end_df = paragraph_df[['paragraph_txt','punctuation_marks','ending_of_a_word','word_order']]

In [379]:
end_df.head(1)

,paragraph_txt,punctuation_marks,ending_of_a_word,word_order
0,나는 다수결을 반대한대 #@문장구분# 왜냐하면 이거를 싫어하는사람도분명히 있을텐데 ...,2,2,3


In [381]:
# jay's data
space_cnt = pd.read_csv('./space_cnt.csv')
spell_rouge = pd.read_csv('./spell_rouge.csv')
score_t = scoret[['paragraph_scoreT_avg']]
score_t.head(1)

,paragraph_scoreT_avg
0,2.142857


In [382]:
end_df = pd.concat([end_df, space_cnt,spell_rouge,score_t], axis=1)

In [383]:
end_df.head(3)

,paragraph_txt,punctuation_marks,ending_of_a_word,word_order,diff,Rouge_l_f1,paragraph_scoreT_avg
0,나는 다수결을 반대한대 #@문장구분# 왜냐하면 이거를 싫어하는사람도분명히 있을텐데 ...,2,2,3,22,0.916001,2.142857
1,교육을 받기를 원하는 사람이 있으면 딱히 받고 싶지 않은 사람이 있을 지도 모른다....,0,0,3,14,1.000001,3.000000
2,남한과 북한의 통일은 너무너무 복잡한 일이라고 생각합니다. #@문장구분# 우리는 북...,0,0,0,14,0.998707,3.000000


In [384]:
# dy's data
dayoung_data = pd.read_csv('./for_mj_original.csv')

In [385]:
dayoung_data.head(3)

,paragraph_txt,essay_level,student_grade_group,org_paragraph,org,org_essay,org_coherence,org_quantity,con_novelty,con_clearance,con,con_prompt,con_description,exp_style,exp_grammar,exp,exp_vocab,essay_grade,essay_main_subject
0,"내가 5~7살쯤에 있던 일이었다. <span>#@문장구분#</span>저는 엄마, ...",3,초등,5,2,0,0,1,2,4,4,4,0,0,4,4,3,초등_4학년,나의 선행
1,나는 생물학자가 되고 싶다.<span>#@문장구분#</span> 어릴적부터 생물관련...,2,고등,3,2,3,2,1,1,4,4,3,0,2,2,4,3,고등_3학년,나의 진로를 위한 노력
2,나는 최근에 ‘스파이더맨:파 프롬 홈’ 이라는 영화를 보았다.<span>#@문장구분...,3,고등,4,2,2,2,1,1,4,4,3,0,1,2,4,3,고등_1학년,영화/독서감상문


In [386]:
# 다영이 데이터에서 paragraph 전처리한것
re_paragraph_dy = pd.read_csv('new_mj.csv')
re_paragraph_dy

,paragraph_txt
0,"내가 5~7살쯤에 있던 일이었다. #@문장구분# 저는 엄마, 누나와 함께 버스를 타..."
1,나는 생물학자가 되고 싶다. #@문장구분# 어릴적부터 생물관련책을 읽으며 생물학을 ...
2,나는 최근에 ‘스파이더맨:파 프롬 홈’ 이라는 영화를 보았다. #@문장구분# 먼저 ...
3,내가 생각하는 참된스승이란 자신의 제자의 잘못된점을 바로 잡아주고 올바른 길로 인도...
4,내가 최근에 본 영화 스파이더맨 파 프롬 홈은 주인공인 스파이더맨이 학교에서 수학여...
...,...
5995,나의 소중함은 나의 친구이다. #@문장구분# 하지만 내일이면 없을지도 모른다. #@...
5996,제가 아주 큰 거북이가 되고 싶습니다. #@문장구분# 영화 니모를 찾아서처럼 물고기...
5997,나는 그냥 평범한 물고기가 되어 보고 싶다. #@문장구분# 아주 특별한 것 없는 평...
5998,저는 만약 바다에서 살 수 있는 기회가 생긴다면 저는 ‘불가사리’가 되고싶습니다. ...


In [387]:
for i in range(len(dayoung_data)):
    dayoung_data['paragraph_txt'][i] = re_paragraph_dy['paragraph_txt'][i]

/opt/conda/envs/lightweight/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [388]:
dayoung_data.head(3)

,paragraph_txt,essay_level,student_grade_group,org_paragraph,org,org_essay,org_coherence,org_quantity,con_novelty,con_clearance,con,con_prompt,con_description,exp_style,exp_grammar,exp,exp_vocab,essay_grade,essay_main_subject
0,"내가 5~7살쯤에 있던 일이었다. #@문장구분# 저는 엄마, 누나와 함께 버스를 타...",3,초등,5,2,0,0,1,2,4,4,4,0,0,4,4,3,초등_4학년,나의 선행
1,나는 생물학자가 되고 싶다. #@문장구분# 어릴적부터 생물관련책을 읽으며 생물학을 ...,2,고등,3,2,3,2,1,1,4,4,3,0,2,2,4,3,고등_3학년,나의 진로를 위한 노력
2,나는 최근에 ‘스파이더맨:파 프롬 홈’ 이라는 영화를 보았다. #@문장구분# 먼저 ...,3,고등,4,2,2,2,1,1,4,4,3,0,1,2,4,3,고등_1학년,영화/독서감상문


In [389]:
# 다영이의 데이터에 빈칸을 만든다.
dayoung_data['punctuation_marks'] = [None]*6000
dayoung_data['ending_of_a_word'] = [None]*6000
dayoung_data['word_order'] = [None]*6000
dayoung_data['diff'] = [None]*6000
dayoung_data['Rouge_l_f1'] = [None]*6000
dayoung_data['paragraph_scoreT_avg'] = [None]*6000

In [338]:
for i in tqdm(range(6000)):
    for j in range(6000):
        if dayoung_data['paragraph_txt'][i] == end_df['paragraph_txt'][j]:
            dayoung_data['punctuation_marks'][i] = end_df['punctuation_marks'][j]
            dayoung_data['ending_of_a_word'][i] = end_df['ending_of_a_word'][j]
            dayoung_data['word_order'][i] = end_df['word_order'][j]
            dayoung_data['diff'][i] = end_df['diff'][j]
            dayoung_data['Rouge_l_f1'][i] = end_df['Rouge_l_f1'][j]
            dayoung_data['paragraph_scoreT_avg'][i] = end_df['paragraph_scoreT_avg'][j]

  0%|          | 0/6000 [00:00<?, ?it/s]/opt/conda/envs/lightweight/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/envs/lightweight/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/envs/lightweight/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [341]:
dayoung_data.to_csv('final_data_v2.csv',index=False)